In [2]:
import os
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
import datetime
from dateutil.relativedelta import relativedelta


In [3]:
def extractDicom (dicomFiles):
    birth_date = ds[0x10, 0x30].value
    examine_date = ds[0x08, 0x20].value
    patient_id = ds[0x10, 0x20].value
    patient_sex = ds[0x10, 0x40].value
    
    #age 
    birth_date_datetime = datetime.datetime.strptime(birth_date, '%Y%m%d')
    examine_date_datetime = datetime.datetime.strptime(examine_date, '%Y%m%d')
    age = abs((birth_date_datetime - examine_date_datetime).days)
    age_in_months = "{:.4}".format(age*0.0328767)

    #sex
    if patient_sex=='M':
        sex='True'
    else :
        sex='False'
        
    return patient_id,age_in_months,sex

In [4]:
dfObj = pd.DataFrame(columns=['id', 'boneage', 'male'])

In [5]:
dfObj.head()

,id,boneage,male


In [7]:
import os
rootdir = "/home/rpitta/hand-atlas/DICOM/combine"
m=0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file
        ds = pydicom.dcmread(filepath)  # plan dataset
        info_id, info_age, info_sex = extractDicom (ds)
        filename = file.replace(".dcm","")
        filename = filename.replace(".DCM","")
        dfObj = dfObj.append({'id': filename, 'boneage': info_age, 'male': info_sex}, ignore_index=True)
        #print (filepath+"-"+str(m))
        m+=1

In [9]:
dfObj.head()

,id,boneage,male
0,7272,139.4,True
1,5524,170.8,True
2,3761,141.2,True
3,4331,194.6,False
4,7261,92.81,False


In [10]:
export_csv = dfObj.to_csv (r'export_dicom.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path